<a href="https://colab.research.google.com/github/iwasakishuto/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments/blob/develop/notebook/colaboratory/microarray2021S_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マイクロアレイ解析 \~2021 Spring Semester\~

- [程研HP](http://ui-tei.rnai.jp/)
- [実習wiki](http://ui-tei.rnai.jp/microarray/doku.php?id=2021)
- [`teilab` Package (Github)](https://github.com/iwasakishuto/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments)
- [`teilab` Documentation](https://iwasakishuto.github.io/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments/index.html)

　今回の実習では、網羅的に遺伝子の発現を解析する手法である**マイクロアレイ**について、

- **wet(実験)**：siRNAを導入した細胞からRNAを抽出し、マイクロアレイ実験を行う。マイクロアレイの原理や操作手順を理解する。
- **dyr(解析)**：全mRNAの変動量を、マイクロアレイデータの特徴を踏まえて解析する。マイクロアレイデータの解析手法を理解する。

の両側面から学んでいただきますが、このNotebookでは、そのうち **dry(解析)** パートに関して、プログラミング言語 [Python](https://www.python.org/) を使って実際の解析の流れを、手を動かしながら学んでいただくことになります。

***
***

#### 一言

　今回dryパートの実習を担当させていただく、程研M1の岩崎（生情11期）です。よろしくお願いします。

　この実習が終わった時には、**全員が「マイクロアレイ解析結構Pythonでできるし、あんなことやこんなことも自分でプログラムを書いて調べてみたい！！」** と思っていただけることを目指して頑張りますので、わからないことがあればなんでも遠慮なく質問してください！！

　また、**「解析手法に絶対はない」** ので、是非積極的に色々な解析を試みて、面白い結果を出してきてください！！！（ぜひ一緒に議論しましょう！）

***
***

## 0. 環境構築

　それでは解析を始めていきましょう！！解析に必要なデータや、各種ツールを揃えていきます。といっても、必要なプログラムはまとめて[パッケージ化しておいた](https://github.com/iwasakishuto/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments)ので、以下のコードを実行するだけで全て揃います。

### 0.1 必要なツールの準備

In [ ]:
! pip install "git+https://github.com/iwasakishuto/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments.git" --ignore-requires-python

ここで

```
ERROR: XXX has requirement YYY==<version> but you'll have YYY <version> which is incompatible.
WARNING: The following packages were previously imported in this runtime
```

のようなエラーが出る分には（確認する限り、今回の実習では）問題ありません。

　ただ、以下のようなメッセージが出ると思うので、言われた通りにruntimeをrestartしましょう。

```
WARNING: The following packages were previously imported in this runtime:
  [matplotlib,mpl_toolkits,numpy,pandas]
You must restart the runtime in order to use newly installed versions.
```

In [ ]:
# 以下のコマンドでエラーが出なければ、準備はOKです！
from teilab.question import ask
ret = ask(
    text="好きな言葉を入力してください。", 
    username="あなたの名前", 
    icon_emoji=":grinning:", 
    icon_url=None, 
    webhook_url=None,
)
ret.ok

　以上で必要なツールは揃いました！！それでは以下で必要なデータのダウンロードを行いましょう！！

### 0.1 必要なデータの準備

　続いて、データの準備に取り掛かります。先ほどインストールしたパッケージを用いてデータのダウンロードを行います。

In [ ]:
# データのダウンロードを取り扱うクラスを定義したので、これを用いて煩わしい作業を省きます。（非本質なので）
from teilab.datasets import TeiLabDataSets
dataset = TeiLabDataSets()

In [ ]:
password1 = ""
password2 = ""
password3 = ""

In [ ]:
path1 = dataset.get_data(password=password1)
path2 = dataset.get_data(password=password2)
path3 = dataset.get_data(password=password3)

In [ ]:
print(path1)
print(path2)
print(path3)

## 1. Pythonについて(Playground)

　Pythonを使うのが初めて、と言う方もいらっしゃると思うので、ここで簡単に説明します。

なお、[東大の数理・情報教育研究センターが提供している入門講座](https://utokyo-ipp.github.io/)が非常に質が良いと評判なので、詳しくはそちらを参照してください。

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
help(ask)

Document: https://iwasakishuto.github.io/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments/teilab.question.html#teilab.question.ask

In [ ]:
dataset.__dir__()

In [ ]:
dir(dataset)

Document: https://iwasakishuto.github.io/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments/teilab.datasets.html#teilab.datasets.TeiLabDataSets

In [ ]:
arr = np.asarray([1,2,3,4,5])
print(arr)

In [ ]:
arr[1:]

In [ ]:
df = pd.DataFrame(data=[
  [1,2,3,4,5],
  [6,7,8,9,10]
], columns=["A","B","C","D","E"])
df.head()

In [ ]:
df[["A","D"]]

In [ ]:
df.E

## 2. データの読み込み

　それでは、ダウンロードしたデータを読み込んでいきます。

In [ ]:
# どこにデータがあるかの確認
dataset.filePaths

In [ ]:
# 読み込まれたデータの内容を可視化する。
dataset.samples.show_groups()

In [ ]:
# データの中身を確認。
! head -n20 {dataset.filePaths[0]}

In [ ]:
# 読み込み方
df = pd.read_csv(filepath_or_buffer=dataset.filePaths[0], sep="\t", header=9)
df.head(3)

## 3. データの統合

　先日 Excel や R で行った解析と同様に、全サンプルのデータを一つにまとめていきましょう。

　行番号とプローブ番号の対応関係は（同じタイミングの実験であれば）どのサンプルも同じであるので、サンプル $X$ のデータの隣にサンプル $Y$ のデータを `concatenate` すれば1枚のワークシート（DataFrame）にまとまります。

　そこで、これを繰り返して全サンプルの `gProcessedSignal` の値を1つのテーブルにまとめていきましょう。

In [ ]:
# 今回は、0-4のデータを使います。
target_filepaths = dataset.filePaths[0:5]
target_conditions = dataset.samples.Condition[0:5]

for fp,cond in zip(target_filepaths, target_conditions):
  print(cond, fp)

### 3.1 アノテーションデータの読み込み

　各プローブ（スポット、フィーチャー）がどういうデータかを説明するデータを、ここでは **「アノテーションデータ」** と呼ぶことにします。なお、同じ実験（マイクロアレイの機械）であれば、この情報はサンプル間で統一されているので、同タイミングに行ったサンプル群の中から一つを適当に選んでそこから抽出します。

※ここで抽出する**アノテーションデータ**は、以下のカラムに書かれている内容です。

In [ ]:
USE_COLS_ANNO  = [
    "FeatureNum", "ControlType", "ProbeName", "SystematicName"
]


|column name|description|
|:-:|:-|
|`FeatureNum`|スポットの番号|
|`ControlType`|<ul><li>positive controlは `1`</li> <li>negative controlは `-1`</li><li>それ以外（解析で用いる）は `0`</li></ul>|
|`ProbeName`|プローブ名|
|`SystematicName`|遺伝子名|

In [ ]:
df_anno = pd.read_csv(filepath_or_buffer=target_filepaths[0], sep="\t", header=9, usecols=USE_COLS_ANNO)
df_anno.head(3)

### 3.2 シグナル値の読み込み

　続いて、各サンプルのシグナル強度( `gProcessedSignal` )のデータを取得します。

　なお、この時 `gIsWellAboveBG` が `0` のものは「（真の）シグナルがバックグラウンドのシグナルよりも低く、信頼できないデータである」ということを意味するため、取り除きます。

In [ ]:
USE_COLS_SYGNAL = [
    "gProcessedSignal", "gIsWellAboveBG"
]

|column name|description|
|:-:|:-|
|`gProcessedSignal`|green(Cy-3)のシグナル強度（＝発現量）|
|`gIsWellAboveBG`|（真の）シグナルがバックグラウンドのシグナルより十分高いか？（＝信頼できるデータか）|

In [ ]:
df_signal = pd.read_csv(filepath_or_buffer=target_filepaths[0], sep="\t", header=9, usecols=USE_COLS_SYGNAL)
df_signal.head(3)

In [ ]:
df_signal.gIsWellAboveBG==1

In [ ]:
# for文で回して0-4全てのデータを読み込みましょう。
df_combined = df_anno.copy(deep=True)
index = set(df_combined.index)
print(f"データ数(before): {len(df_combined)}")

for fp,cond in zip(target_filepaths, target_conditions):
  df_signal = pd.read_csv(fp, sep="\t", header=9, usecols=USE_COLS_SYGNAL)
  index = index & set(df_signal[(df_signal.gIsWellAboveBG==1)].index)
  df_combined = pd.concat([df_combined, df_signal[["gProcessedSignal"]].rename(columns={"gProcessedSignal" : cond})], axis=1)

df_filtered_1 = df_combined.loc[index, :]
print(f"データ数(after) : {len(df_filtered_1)}")

　また、`ControlType` の値が $\pm1$ のものはコントロールであるため、`0` のもののみ取り出します。

In [ ]:
print("データ数(before):", len(df_filtered_1))
df_filtered_2 = df_filtered_1[df_filtered_1.ControlType == 0]
print("データ数(after) :", len(df_filtered_2))

In [ ]:
# インデックスを振り直す。
df_filtered = df_filtered_2.reset_index(drop=True)
df_filtered.head(5)

### 3.3 アノテーションデータの追加

　このままだと `GeneName` がわからないので、その列を追加します。

In [ ]:
import os

In [ ]:
path3

In [ ]:
os.listdir(path3)

In [ ]:
df_correspondence = pd.read_csv(os.path.join(path3, "072363_D_AA_20200924.txt"), sep="\t", usecols=["ProbeID", "GeneSymbol"])
df_correspondence.head(3)

In [ ]:
df_filtered = pd.merge(left=df_filtered, right=df_correspondence, left_on="ProbeName", right_on="ProbeID", how="left")
df_filtered.head(3)

In [ ]:
df_filtered.info()

In [ ]:
# 欠損値の処理をする。
df_filtered["ProbeID"]    = df_filtered["ProbeID"].fillna("Not Found")
df_filtered["GeneSymbol"] = df_filtered["GeneSymbol"].fillna("unknown")
df_filtered.head(3)

***
```python
# データをGoogleDriveに保存したい場合は、以下のコードを走らせてください。
from google.colab import drive
drive.mount('/content/drive')
df_filtered.to_excel("microarray_filtered.xlsx", index=False)
```
***

## 4. データの前処理

　無事にデータがダウンロードできたので、実験上のバイアス等を取り除くためにデータの前処理を行います。（ここでは省略します。）

In [ ]:
df_filtered.columns

## 5. 可視化

　`matplotlib` や `plotly` などを用いて、解析結果を可視化します。

In [ ]:
X = [1,2,3, 4, 5, 6, 7, 8, 9, 10]
Y = [1,4,9,16,25,36,49,64,81,100]

### 5.1 `matplotlib`

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X,Y)

In [ ]:
plt.scatter(X,Y)


### 5.2 `plotly`

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
px.line(x=X,y=Y, width=400, height=300)

In [ ]:
px.scatter(x=X, y=Y, width=400, height=300)

## 6. 解析

　ここでは、XYプロットとMAプロットを図示し、シグナル強度の分布を調べます。

In [ ]:
df_filtered["siVIM-270(2'OMe2-5)"]

### 6.1 X-Y プロット

- サンプル $X$ の `gProcessedSignal` （$\log_2(\text{gProcessedSignal})$） の値を横軸
- サンプル $Y$ の `gProcessedSignal` （$\log_2(\text{gProcessedSignal})$） の値を縦軸

にプロットしたものを **X-Yプロット** と呼びます。

In [ ]:
np.log2([2,4,8,16])

### 6.2 MAプロット

- $log_2(Y/X)$ を縦軸 (Minus)
- $log_{10}(XY)$ を横軸 (Average)

にプロットしたものを **M-Aプロット** と呼びます。

In [ ]:
np.log10([10,100,1000])

## 7. 発展

　より良いデータ解析のためには、以下のことが欠かせません。

- プローブのSummarization
- サンプル間のNormalization
- サンプルのSummarization

## 7. 特定の塩基配列を持つ遺伝子の発現量の変化を調べる

　入力した配列を3'UTRにもつ遺伝子（アクセッション番号）のリストを表示するページ（[seedmatch](http://atlas.RNAi.jp/seedmatch/)）を用いて、特定の配列を3'UTRに持つ遺伝子群を見てみましょう。

### 7.1 シード領域と相補的な配列を持つ遺伝子について調べる

siRNAのガイド鎖の5'末端から2-8塩基目の7塩基（シード領域と呼ばれます。今回は `UGAACUC`）と相補的な配列（`GAGTTCA`）を3'UTRにもつmRNAsを検索してみましょう。

In [ ]:
df_matched_mRNAs = pd.read_csv("seedmatch.txt", sep="\t")
df_matched_mRNAs.columns = ["SystematicName", "NumHits"]

In [ ]:
# seedmatchで検索したデータと紐付ける。
df_is_matched = pd.merge(df_filtered, df_matched_mRNAs, on="SystematicName", how="left").fillna(0)
df_is_matched.head(3)

In [ ]:
# 累積度数曲線を描くために、ソートする。
df_is_matched["log2(RNA/mock)"] = np.log2(df_is_matched["siVIM-270"]/df_is_matched["mock"])
df_is_matched = df_is_matched.sort_values(by="log2(RNA/mock)").reset_index(drop=False)
df_is_matched.head(3)

### 7.2 さらに遊んでみる。



　ここから先は、みなさんが興味を持った点について、思う存分データで遊んでいただく時間です。

- マッチするシードの数（`NumHits`）って、多い方が抑制されてる？？
- そもそもなんで2-8の7merでオフターゲット効果が起きるの？？
  - 1-7や3-9は？？
  - 6merや9merは？？
  - Argonauteタンパク質と結合（loading）し、RISC(RNA induced silencing complex)を形成するが、構造的に…
- 統計的に有意だと言える？？

面白そうなことについては、積極的に調べて見てください！！

※ 以下に、使えそうなツールを用意しておきました。ぜひ解析に役立ててください！！

In [ ]:
from teilab.seedmatch import get_matched_mRNAs

In [ ]:
df_matched_mRNAs = get_matched_mRNAs("GAGTTCA")
df_matched_mRNAs.head(5)